In [1]:
#Import necessary modules inlcuded Tensorflow
import pandas as pd
import tensorflow as tf
#import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Read in File
data = pd.read_csv("balanced_pop_unpop_features.csv")

In [3]:
#Remove track name column
del data["Unnamed: 0"]

In [4]:
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,label
0,0.21400,0.666,178242,0.677,0.000000,2,0.0979,-5.743,0.0326,100.014,4,0.178,1
1,0.01340,0.807,183750,0.916,0.000012,0,0.0787,-3.282,0.2260,127.973,4,0.651,1
2,0.00162,0.791,279507,0.615,0.000065,6,0.0812,-6.149,0.0667,128.017,4,0.393,1
3,0.76300,0.707,275227,0.709,0.000000,11,0.2740,-3.979,0.3400,89.094,4,0.501,1
4,0.57000,0.629,250173,0.572,0.000000,5,0.1920,-7.733,0.0387,100.015,4,0.386,1


In [5]:
#Shuffle dataset
for i in range(10): data = data.sample(frac=1)

In [6]:
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,label
302,0.33300,0.439,183413,0.802,0.000000,7,0.377,-3.644,0.0540,168.032,4,0.7250,1
247,0.01240,0.669,227280,0.855,0.000000,0,0.335,-3.035,0.0499,120.045,4,0.6480,1
329,0.00713,0.533,263645,0.671,0.000106,1,0.120,-4.464,0.0334,111.044,4,0.0449,1
859,0.04710,0.533,226080,0.722,0.000009,0,0.242,-6.817,0.0341,107.001,4,0.3960,0
981,0.14500,0.291,215920,0.400,0.000019,2,0.259,-10.615,0.0306,83.112,1,0.0935,0


In [7]:
#About a 70-30 split between training and testing 
train = data.iloc[:750]
test = data.iloc[750:]

In [8]:
train.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,label
302,0.33300,0.439,183413,0.802,0.000000,7,0.377,-3.644,0.0540,168.032,4,0.7250,1
247,0.01240,0.669,227280,0.855,0.000000,0,0.335,-3.035,0.0499,120.045,4,0.6480,1
329,0.00713,0.533,263645,0.671,0.000106,1,0.120,-4.464,0.0334,111.044,4,0.0449,1
859,0.04710,0.533,226080,0.722,0.000009,0,0.242,-6.817,0.0341,107.001,4,0.3960,0
981,0.14500,0.291,215920,0.400,0.000019,2,0.259,-10.615,0.0306,83.112,1,0.0935,0


In [9]:
print(train.shape,test.shape)

(750, 13) (322, 13)


In [10]:
#Split entries in train_x: features and train_y: labels
train_x = train.iloc[:,:12]
train_y = train.iloc[:,12:].astype(int)

test_x = test.iloc[:,:12]
test_y = test.iloc[:,12:].astype(int)

#Take Z-score
train_x = (train_x-train_x.mean())/(train_x.std())
test_x = (test_x-test_x.mean())/(test_x.std())

In [11]:
train_x.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
302,0.773343,-1.441626,-0.929910,0.709403,-0.117480,0.470931,1.298660,0.938461,-0.444393,1.614429,0.075590,0.942037
247,-0.757951,0.201424,0.128045,1.021363,-0.117480,-1.439723,1.013773,1.184708,-0.486950,-0.062375,0.075590,0.598251
329,-0.783122,-0.770119,1.005071,-0.061668,-0.115482,-1.166773,-0.444575,0.606895,-0.658215,-0.376896,0.075590,-2.094443
859,-0.592212,-0.770119,0.099104,0.238520,-0.117311,-1.439723,0.382953,-0.344535,-0.650949,-0.518170,0.075590,-0.526867
981,-0.124608,-2.498893,-0.145928,-1.656785,-0.117130,-0.893822,0.498264,-1.880248,-0.687278,-1.352921,-10.554218,-1.877456


In [12]:
#Create feature column for net 
feature_columns = []
for i in train_x.columns:
    feature_columns.append(tf.feature_column.numeric_column(key=i))

In [13]:
#Lil preview 
for i in feature_columns:
    print(i)
    print("")

_NumericColumn(key='acousticness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='danceability', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='duration_ms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='energy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='instrumentalness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='key', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='liveness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='loudness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='speechiness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='tempo', shape=(1,), default_value=None, dtype=t

In [14]:
#Make estimator/ Model
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns
                                        , hidden_units=[5],
                                        n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2qi3xv2o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10d939160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

In [16]:
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y,300),
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2qi3xv2o/model.ckpt.
INFO:tensorflow:loss = 233.11339, step = 1
INFO:tensorflow:global_step/sec: 234.548
INFO:tensorflow:loss = 204.14357, step = 101 (0.427 sec)
INFO:tensorflow:global_step/sec: 286.056
INFO:tensorflow:loss = 201.40352, step = 201 (0.350 sec)
INFO:tensorflow:global_step/sec: 300.621
INFO:tensorflow:loss = 194.06752, step = 301 (0.333 sec)
INFO:tensorflow:global_step/sec: 302.512
INFO:tensorflow:loss = 195.32372, step = 401 (0.331 sec)
INFO:tensorflow:global_step/sec: 301.079
INFO:tensorflow:loss = 187.56506, step = 501 (0.332 sec)
INFO:tensorflow:global_step/sec: 299.255
INFO:tensorflow:loss = 194.2115, step = 601 (0.334 sec)
INFO:tensorflo

In [17]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [18]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 3000))



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-14-20:20:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2qi3xv2o/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-14-20:20:59
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.49068323, accuracy_baseline = 0.50621116, auc = 0.50028944, auc_precision_recall = 0.5020812, average_loss = 0.7457761, global_step = 1000, label/mean = 0.4937888, loss = 240.13991, precision = 0.48447204, prediction/mean = 0.50194633, recall = 0.49056605


In [19]:
eval_result

{'accuracy': 0.49068323,
 'accuracy_baseline': 0.50621116,
 'auc': 0.50028944,
 'auc_precision_recall': 0.5020812,
 'average_loss': 0.7457761,
 'global_step': 1000,
 'label/mean': 0.4937888,
 'loss': 240.13991,
 'precision': 0.48447204,
 'prediction/mean': 0.50194633,
 'recall': 0.49056605}

In [20]:
test_x.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
633,-0.717737,0.458499,-0.253128,1.756477,-0.100006,0.342645,1.156925,1.660456,1.025009,-0.444433,0.186002,0.344673
544,-0.502073,-0.368556,-0.799667,0.854896,-0.099573,-1.536482,0.204970,0.223395,-0.556040,0.794611,0.186002,0.950667
134,-0.644133,-1.040538,-0.128400,-0.071218,-0.099910,0.611091,0.174507,-0.296025,-0.197696,1.883591,0.186002,-1.431674
848,-0.387807,0.517574,-0.609492,1.339419,-0.100547,0.611091,0.463901,1.168177,-0.001485,-1.136622,0.186002,1.593667
214,-0.600897,1.241247,1.453026,0.296775,-0.100547,-1.268035,1.217850,0.173325,-0.236938,-0.882003,0.186002,-0.353839


In [21]:
#Look at results of net preidctions compared to actual label
test_x_np = test_x.values.transpose()


predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=dict(test_x),
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

for i,j in zip(predicted_classes,test_y.label):
    print(i,j)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2qi3xv2o/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'1'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'1'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'1'] 1
[b'0'] 1
[b'1'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'1'] 1
[b'0'] 0
[b'0'] 1
[b'1'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'1'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'1'] 1
[b'0'] 0
[b'0'] 1
[b'1'] 1
[b'1'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'1'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'1'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'1'] 1
[b'0

/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]
